In [3]:
from google.colab import drive
drive.mount('/content/drive')

Mounted at /content/drive


In [4]:
%cd drive/MyDrive


/content/drive/MyDrive


In [5]:
import os
import cv2
import pandas as pd

from tqdm import tqdm
from torch import optim
from datetime import datetime
from torchvision.utils import save_image
from torchvision.transforms import Resize, functional

import PIL
import random
import torchvision

from pathlib import Path
from torch.utils.data import Dataset, DataLoader

import torch
import torch.nn as nn
import torch.nn.functional as f

In [6]:
# hacks for time-saving

torch.backends.cudnn.benchmark = True

torch.autograd.profiler.emit_nvtx(False)
torch.autograd.profiler.profile(False)
torch.autograd.set_detect_anomaly(False)

device = torch.device("cuda" if torch.cuda.is_available() else "cpu")

In [7]:
os.getcwd()

'/content/drive/MyDrive'

In [8]:
!nvidia-smi

Wed Sep 15 08:45:09 2021       
+-----------------------------------------------------------------------------+
| NVIDIA-SMI 470.63.01    Driver Version: 460.32.03    CUDA Version: 11.2     |
|-------------------------------+----------------------+----------------------+
| GPU  Name        Persistence-M| Bus-Id        Disp.A | Volatile Uncorr. ECC |
| Fan  Temp  Perf  Pwr:Usage/Cap|         Memory-Usage | GPU-Util  Compute M. |
|                               |                      |               MIG M. |
|===============================+======================+======================|
|   0  Tesla K80           Off  | 00000000:00:04.0 Off |                    0 |
| N/A   53C    P8    32W / 149W |      3MiB / 11441MiB |      0%      Default |
|                               |                      |                  N/A |
+-------------------------------+----------------------+----------------------+
                                                                               
+-------

In [9]:
torch.cuda.empty_cache()

In [10]:
def get_train_test_paths(test_ratio: float = 0.2):
    # extract the data from the dataset folder
    files = [file_name for file_name in
             Path(os.getcwd() + os.sep + 'Water Bodies Dataset' + os.sep + 'Images').rglob("*.jpg")]
    # randomize the order of the data
    random.shuffle(files)
    # separate test and train files
    first_train = int(test_ratio * len(files))
    test_path = files[:first_train]
    train_path = files[first_train:]
    return train_path, test_path


def get_mask_path(file_path):
    # gets source image path, returns mask path
    file_path = str(file_path).replace('Images', 'Masks')
    return file_path


class WaterDataset(Dataset):
    def __init__(self, path_list, transform_source=None, transform_both=None):
        self.sources = path_list
        self.transform_source = transform_source
        self.transform_both = transform_both

    def __len__(self):
        return len(self.sources)

    def __getitem__(self, index):
        img_path = self.sources[index]
        source = functional.to_tensor(PIL.Image.open(img_path))
        label = functional.to_tensor(PIL.Image.open(get_mask_path(img_path)).convert('L'))

        if self.transform_source:
            source = self.transform_source(source)
        if self.transform_both:
            source = self.transform_both(source)
            label = self.transform_both(label)
            label = (label < 0.5).float()

            assert len(label.unique()) <= 2, "threshold didn't work"

        return source, label


def get_train_test_loaders(batch_size, length):
    train_path, test_path = get_train_test_paths()
    train_loader = DataLoader(dataset=WaterDataset(train_path,
                                                   transform_both=torchvision.transforms.Resize((length, length))),
                              batch_size=batch_size,
                              pin_memory=True,
                              num_workers=2,
                              shuffle=True)
    test_loader = DataLoader(dataset=WaterDataset(test_path,
                                                  transform_both=torchvision.transforms.Resize((length, length))),
                             batch_size=batch_size,
                             pin_memory=True,
                             num_workers=2)

    return train_loader, test_loader

In [17]:
class Hidden1(nn.Module):
    # define the model
    def __init__(self, length, hidden_size, activation):
        super().__init__()
        self.activation = activation
        self.length = length

        self.flat = nn.Flatten()
        self.fc1 = nn.Linear(length**2 * 3, hidden_size)
        self.fc2 = nn.Linear(hidden_size, length**2 * 2)  # 2 for each class

    # set activation functions for the layers
    def forward(self, x):
        x = self.flat(x)
        x = self.activation(self.fc1(x))
        x = self.activation(self.fc2(x))
        x = x.reshape(x.size(0) * self.length**2, 2)
        return x


class Hidden2(nn.Module):
    # define the model
    def __init__(self, length, hidden_size, activation):
        super().__init__()
        self.activation = activation
        self.length = length

        self.flat = nn.Flatten()
        self.fc1 = nn.Linear(length**2 * 3, hidden_size)
        self.fc2 = nn.Linear(hidden_size, hidden_size)
        self.fc3 = nn.Linear(hidden_size, length**2 * 2)  # 2 for each class

    # set activation functions for the layers
    def forward(self, x):
        x = self.flat(x)
        x = self.activation(self.fc1(x))
        x = self.activation(self.fc2(x))
        x = self.activation(self.fc3(x))
        x = x.reshape(x.size(0) * self.length**2, 2)
        return x


class Conv1(nn.Module):
    # define the model
    def __init__(self, length, hidden_size, activation, kernel_size: int = 3):
        super().__init__()
        self.activation = activation
        self.length = length
        self.kernel_size = kernel_size  # expected odd kernel_size
        self.hidden_size = hidden_size

        self.conv1 = nn.Conv2d(3, 3, self.kernel_size, padding=int((self.kernel_size - 1) / 2))
        self.fc1 = nn.Linear(length**2 * 3, self.hidden_size)
        self.fc2 = nn.Linear(self.hidden_size, self.hidden_size)
        self.fc3 = nn.Linear(self.hidden_size, length**2 * 2)

    def forward(self, x):
        x = self.activation(self.conv1(x))
        x = f.max_pool2d(x, kernel_size=self.kernel_size, stride=1, padding=int((self.kernel_size - 1) / 2))
        x = torch.flatten(x, 1)
        x = self.activation(self.fc1(x))
        x = self.activation(self.fc2(x))
        x = self.fc3(x)
        x = x.reshape(x.size(0) * self.length**2, 2)
        return x


class Conv2(nn.Module):
    # define the model
    def __init__(self, length, hidden_size, activation, kernel_size: int = 3):
        super().__init__()
        self.activation = activation
        self.length = length
        self.kernel_size = kernel_size  # expected odd kernel_size
        self.hidden_size = hidden_size

        self.conv1 = nn.Conv2d(3, 3, self.kernel_size, padding=int((self.kernel_size - 1) / 2))
        self.conv2 = nn.Conv2d(3, 3, self.kernel_size, padding=int((self.kernel_size - 1) / 2))
        self.fc1 = nn.Linear(length**2 * 3, self.hidden_size)
        self.fc2 = nn.Linear(self.hidden_size, self.hidden_size)
        self.fc3 = nn.Linear(self.hidden_size, length**2 * 2)

    def forward(self, x):
        x = self.activation(self.conv1(x))
        x = f.max_pool2d(x, kernel_size=self.kernel_size, stride=1, padding=int((self.kernel_size - 1) / 2))
        x = self.activation(self.conv2(x))
        x = f.max_pool2d(x, kernel_size=self.kernel_size, stride=1, padding=int((self.kernel_size - 1) / 2))
        x = torch.flatten(x, 1)
        x = self.activation(self.fc1(x))
        x = self.activation(self.fc2(x))
        x = self.fc3(x)
        x = x.reshape(x.size(0) * self.length**2, 2)
        return x


class Conv3(nn.Module):
    # define the model
    def __init__(self, length, hidden_size, activation, kernel_size: int = 3):
        super().__init__()
        self.activation = activation
        self.length = length
        self.kernel_size = kernel_size  # expected odd kernel_size
        self.hidden_size = hidden_size

        self.conv1 = nn.Conv2d(3, 3, self.kernel_size, padding=int((self.kernel_size - 1) / 2))
        self.conv2 = nn.Conv2d(3, 3, self.kernel_size, padding=int((self.kernel_size - 1) / 2))
        self.conv3 = nn.Conv2d(3, 3, self.kernel_size, padding=int((self.kernel_size - 1) / 2))
        self.fc1 = nn.Linear(length**2 * 3, self.hidden_size)
        self.fc2 = nn.Linear(self.hidden_size, self.hidden_size)
        self.fc3 = nn.Linear(self.hidden_size, length**2 * 2)

    def forward(self, x):
        x = self.activation(self.conv1(x))
        x = f.max_pool2d(x, kernel_size=self.kernel_size, stride=1, padding=int((self.kernel_size - 1) / 2))
        x = self.activation(self.conv2(x))
        x = f.max_pool2d(x, kernel_size=self.kernel_size, stride=1, padding=int((self.kernel_size - 1) / 2))
        x = self.activation(self.conv3(x))
        x = f.max_pool2d(x, kernel_size=self.kernel_size, stride=1, padding=int((self.kernel_size - 1) / 2))
        x = torch.flatten(x, 1)
        x = self.activation(self.fc1(x))
        x = self.activation(self.fc2(x))
        x = self.fc3(x)
        x = x.reshape(x.size(0) * self.length**2, 2)
        return x

In [ ]:
def save_prediction(prediction, index, name):
    # convert two-valued pixels to single-max-value
    prediction = torch.argmax(prediction, dim=1)
    # reshape the flattened prediction to a matrix
    prediction = prediction.reshape(100, 100)
    # get original size
    source_path = f'{os.getcwd()}{os.sep}Water Bodies Dataset{os.sep}Images{os.sep}water_body_{index}.jpg'
    width, height = cv2.imread(source_path).shape
    # resize prediction to original source size
    prediction = Resize(prediction, size=(width, height))
    prediction_path = f'{os.getcwd()}{os.sep}Deep Images{os.sep}{name}_{index}.jpg'
    # save the prediction
    save_image(prediction, prediction_path)


def get_n_params(model):
    """https://discuss.pytorch.org/t/how-do-i-check-the-number-of-parameters-of-a-model/4325/6"""
    pp = 0
    for p in list(model.parameters()):
        nn = 1
        for s in list(p.size()):
            nn = nn * s
        pp += nn
    return pp


def get_img_index(path):
    index = str(path).split('_')[-1].split('.')[0]
    return index


def fit_model(model, model_parameters, loss_function, optimizer, batch_size, image_normalized_length, num_of_epochs):
    # retrieve train and test files
    train_loader, test_loader = get_train_test_loaders(batch_size, image_normalized_length)
    # if GPU is available, prepare it for heavy calculations
    device = torch.device("cuda" if torch.cuda.is_available() else "cpu")
    # assign the model
    model = model(*model_parameters).to(device)
    # assign the loss function
    criterion = loss_function()
    # set an optimizer
    optimizer = optimizer(model.parameters(), lr=3e-4)
    for epoch in range(1, num_of_epochs + 1):
        # MODEL TRAINING
        model.train()
        # start counting epoch duration
        epoch_start = datetime.now()
        # initiate epoch loss
        epoch_loss = 0
        # iterate through all data pairs
        for image, mask in tqdm(train_loader):
            # convert input pixel to tensor
            x = image.float().to(device)
            # convert target to tensor
            tag = mask.flatten().long().to(device)
            # reset all gradients
            optimizer.zero_grad()
            # save current prediction
            prediction = model(x)
            # activate loss function, calculate loss
            loss = criterion(prediction, tag)
            # back propagation
            loss.backward()
            optimizer.step()
            # update epoch loss
            epoch_loss += loss.item()
        # stop counting epoch duration
        epoch_end = datetime.now()
        epoch_seconds = (epoch_end - epoch_start).total_seconds()
        # MODEL EVALUATION
        model.eval()
        # collect predicted results and real results
        total_predicted_positive, total_true_positive, total_false_negative, \
        total_true_prediction, total_false_prediction = 0, 0, 0, 0, 0
        with torch.no_grad():
            for x, y in tqdm(test_loader):
                x = x.to(device)
                y = y.flatten().to(device)
                probabilities = model(x)
                prediction = torch.argmax(probabilities, dim=1)

                predicted_positive = (prediction == 1).sum().item()
                true_positive = ((prediction == 1) & (y == 1)).sum().item()
                false_negative = ((prediction == 0) & (y == 1)).sum().item()

                total_predicted_positive += predicted_positive
                total_true_positive += true_positive
                total_false_negative += false_negative
                total_true_prediction += (prediction == y).sum().item()
                total_false_prediction += (prediction != y).sum().item()
        # calculate accuracy and f1 score
        recall = total_true_positive / (total_true_positive + total_false_negative)
        precision = total_true_positive / total_predicted_positive
        f1 = (2 * precision * recall) / (precision + recall)
        accuracy = total_true_prediction / (total_true_prediction + total_false_prediction)
        # append results to csv file
        df = pd.DataFrame({'Model Name': [model.__class__.__name__],
                           'Iteration': [epoch],
                           'Input Image Length': [image_normalized_length],
                           'Hidden Layer Size': [hidden_layer_size],
                           'Batch Size': [batch_size],
                           'Activation Function': [str(model_parameters[2].__name__)],
                           'Optimizer': [str(type(optimizer))],
                           'Loss Function': [str(loss_function)],
                           'Loss': [epoch_loss],
                           'Recall': [recall],
                           'Precision': [precision],
                           'F1': [f1],
                           'Accuracy': [accuracy],
                           'Iteration Training Seconds': [epoch_seconds]})
        df.to_csv('Water_Bodies_Results.csv', index=False, mode='a', header=False)
        print(df)

    ### SAVE A PREDICTION ###
    # with torch.no_grad():
    #     for i, image, mask in enumerate(test_loader):
    #         name = ''
    #         x = image.float().to(device)
    #         tag = mask.flatten().long().to(device)
    #         prediction = model(x)
    #         file_name, _ = test_loader.dataset.samples[i]
    #         file_index = get_img_index(file_name)
    #         save_prediction(prediction, file_index, name)

    # clean memory of current model on exit
    torch.cuda.empty_cache()


if __name__ == '__main__':
    models = (Hidden1,)
    activation_funcs = (f.relu, f.leaky_relu)
    hidden_layer_sizes = (4000, 3000, 2000)

    optimizer = optim.Adam
    loss_func = nn.CrossEntropyLoss
    image_normalized_length = 100
    batch_size = 16
    num_of_epochs = 10
    kernel_size = 3

    # train models with varying hyperparameters
    for model in models:
        for activation_func in activation_funcs:
            for hidden_layer_size in hidden_layer_sizes:
                model_parameters = (image_normalized_length, hidden_layer_size, activation_func)
                fit_model(model, model_parameters, loss_func, optimizer,
                          batch_size, image_normalized_length, num_of_epochs)

In [13]:
### WIPE RESULTS FILE ###
# df = pd.DataFrame({'Model Name': [],
#                    'Iteration': [],
#                    'Input Image Length': [],
#                    'Hidden Layer Size': [],
#                    'Batch Size': [],
#                    'Activation Function': [],
#                    'Optimizer': [],
#                    'Loss Function': [],
#                    'Loss': [],
#                    'Recall': [],
#                    'Precision': [],
#                    'F1': [],
#                    'Accuracy': [],
#                    'Iteration Training Seconds': []})
# df.to_csv('Water_Bodies_Results.csv', index=False, header=True)